# Inteligencia Artificial
# Clase 33 - Detección de Objetos con YOLO

En esta notebook vamos a hacer detección de objetos en imágenes, videos y *feeds* de webcam utilizando [YOLO v4](https://arxiv.org/abs/2004.10934), la versión de YOLO publicada en abril del 2020 por Alexey Bochkovskiy et al. 
Trabajaremos con la implementación original de los autores del modelo, la cual se desarrolló con el *framework* [Darknet](https://github.com/pjreddie/darknet), creado por Joseph Redmon (autor de la primera versión de YOLO).

<img src="https://miro.medium.com/max/1396/1*Co8xD0IWPaBiWr-Xfu38dw.jpeg">

El código de esta notebook se basa mayormente en los ejemplos de [The AI Guy](https://github.com/theAIGuysCode), mencionados por Alexey Bochkovskiy en su [publicación de divulgación en Medium](https://medium.com/@alexeyab84/yolov4-the-most-accurate-real-time-neural-network-on-ms-coco-dataset-73adfd3602fe).


## Setup

Para poder utilizar la implementación de Darknet de YOLO v4, seguiremos los siguientes pasos:

1. Clonar el [repositorio oficial](https://github.com/AlexeyAB/darknet).
2. Ajustar el Makefile para habilitar OpenCV y el uso de la GPU.
3. Construir Darknet.
4. Descargar los pesos del modelo preentrenado en el dataset [COCO](https://cocodataset.org/#explore) (acrónimo de *Common Objects in Context*).

In [ ]:
# Clonamos el repositorio oficial
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Ajustamos el Makefile para habilitar OpenCV y el uso de la GPU
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# Construimos Darknet
!make

In [ ]:
# Descargamos los pesos del modelo preentrenado en COCO
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

¡Ya está todo listo para comenzar a detectar objetos en imágenes!

##  Corriendo el modelo en imágenes estáticas

Hecho el setup, ya podemos utilizar YOLO para hacer detección de objetos en imágenes. Corremos el modelo con el siguiente comando:

```bash
# En Linux
./darknet detector test <path to .data file> <path to config> <path to weights> <path to image>

# En Windows
darknet.exe detector test <path to .data file> <path to config> <path to weights> <path to image>
```

Darknet provee algunas imágenes precargadas en el directorio darknet/data, así que comenzamos explorándolas:

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/dog.jpg

Una vez finalizada la inferencia, obtenemos información acerca de los tiempos de *runtime*, las clases detectadas y la confianza en la detección (expresada como una probabilidad).

**Nota:** Luego de correr la detección, OpenCV no puede abrir las imágenes al instante en Google Colab, por lo que definimos una función utilitaria para mostrar las predicciones del modelo:

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Definimos una función utilitaria para mostrar las predicciones del modelo
def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
# La detección más reciente se guarda por defecto como 'predictions.jpg'
imShow('predictions.jpg')

¡Excelente! El modelo detecta y localiza objetos pertenencientes a distintas clases presentes en la imagen, incluso cuando se encuentran en diferentes escalas espaciales e incluso con cierta superposición entre ellos (como en el caso del perro y la bicicleta).

Veamos otro ejemplo:

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg

In [ ]:
imShow('predictions.jpg')

¡Muy bien! El modelo detecta correctamente la persona y los animales presentes en la imagen.

Veamos ahora qué ocurre cuando probamos con una imagen más cargada de contenido... 

In [ ]:
# Montamos el Google Drive para acceder facilmente a los archivos que necesitamos
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Creamos un symbolic link para poder llamar al path "/content/gdrive/My\ Drive/" como /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

**Nota:** para que el código a continuación funcione, debemos subir a nuestro Drive el contenido del archivo `yolov4.zip`.

In [ ]:
%cd darknet
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights /mydrive/yolov4/imgs/street.jpg

In [ ]:
imShow('predictions.jpg')

¡Increíble! Pese a la multitud, el modelo puede resolver más que bien la localización de distintas personas, e incluso logra detectar otros objetos como el bolso de la muchacha o algunas luces de tránsito.

Veamos ahora qué tan bien puede predecir sobre una pintura en lugar de una imagen del mundo real...

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights /mydrive/yolov4/imgs/frida.png

In [ ]:
imShow('predictions.jpg')

¡Bastante bien! El modelo detecta correctamente a Frida Kahlo y su gato, aunque también comete dos errores:

- Omite la detección del mono. 
- Detecta erróneamente una cama completamente ausente en la obra.

El primer error en realidad no lo es tanto... aunque hay perros, gatos, caballos y osos en el dataset de entrenamiento (y otros tantos animales), ¡los monos no forman parte de las clases de COCO! Por lo tanto, es natural que el modelo no pueda detectarlos en las imágenes. Para lograr la detección de ésta u otras clases, tendríamos que reentrenar al modelo sobre un dataset *custom*, algo que haremos más adelante.

Por su parte, el segundo error podría corregirse exigiéndole al modelo mayor confianza en la detección. Por defecto, el modelo retornará cualquier clase detectada con al menos un 25% de confianza. Podemos regular el umbral de confianza mínima exigida introduciendo el *flag* `-thresh` en el comando de ejecución del detector:

In [ ]:
# Incrementamos el umbral de confianza mínima a 0.5
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights /mydrive/yolov4/imgs/frida.png -thresh 0.5

In [ ]:
imShow('predictions.jpg')

¡Mucho mejor! La detección de una cama inexistente ya no entorpece a la predicción.

Veamos ahora qué ocurre cuando corremos el detector con un umbral excesivamente bajo, del orden del 5%:

In [ ]:
# Reducimos el umbral de confianza mínima a 0.05
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/dog.jpg -thresh 0.05

In [ ]:
imShow('predictions.jpg')

Observamos que si usáramos un umbral de confianza excesivamente bajo, esto introduciría mucho ruido en las predicciones, ya que se localizarían objetos que en realidad no están presentes en la imagen y, al mismo tiempo, podríamos obtener más de una etiqueta para el mismo objeto, con algunas de ellas completamente erradas.

Si queremos tener el detalle de las coordenadas de las *bounding boxes* detectadas, podemos generar una impresión con mayor información al respecto introduciendo el *flag* `-ext_output` al correr el modelo:

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/dog.jpg -thresh 0.5 -ext_output

Para que el modelo genere predicciones sobre un conjunto de imágenes, debemos contar con un archivo `.txt` que contenga los *path* de cada imagen. Los resultados serán exportados en un archivo `.json`, el cual se especifica luego del *flag* `-out`.

In [ ]:
# El flag -dont_show evita la impresión de warnings y el display de las imágenes (esto último, en Colab no ocurre de cualquier forma)
# -out especifica el archivo donde se graba la salida
# Luego del < se indica la ruta del archivo .txt con los paths de las imágenes a predecir
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -thresh 0.5 -ext_output -dont_show -out result.json < /mydrive/yolov4/imgs/images.txt

Alternativamente, podemos guardar el resultado en un archivo `.txt`, en el cual podemos guardar información más detallada del proceso de inferencia.

In [ ]:
# Notemos que cambiamos la sintaxis: se reemplaza el -out result.json por un > result.txt
# De esta manera, en el archivo .txt se puede guardar otra información (no estrucutarada con el formato del .json) 
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -thresh 0.5 -ext_output -dont_show > result.txt < /mydrive/yolov4/imgs/images.txt

##  Corriendo el modelo en videos

Ya vimos la potencia de YOLO v4 para detectar objetos en imágenes. Ahora veremos que esta misma técnica también nos permite procesar videos. Para hacerlo con la implementación oficial de Darknet, sólo tenemos que modificar ligeramente el comando de ejecución:

```bash
# En Linux
./darknet detector demo <path to .data file> <path to config> <path to weights> <path to video> <path to video> [-out_filename <path to video with predictions>]

# En Windows
darknet.exe detector demo <path to .data file> <path to config> <path to weights> <path to video> <path to video> [-out_filename <path to video with predictions>]
```

Veamos un ejemplo:

In [ ]:
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights /mydrive/yolov4/videos/test.mp4 -dont_show -i 0 -out_filename /mydrive/yolov4/videos/results.avi

¡Excelente! En poco tiempo, el modelo se encargó de generar un nuevo video en el que se incluyen las *bounding boxes* de las predicciones.

##  Corriendo el modelo con *feeds* de webcam

Podemos hacer inferencia en tiempo real alimentando al modelo con el *feed* de una webcam. El comando de ejecución para hacer esto es:

```bash
# En Linux
./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -c 0

# En Windows
darknet.exe detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -c 0
```

In [ ]:
#!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -c 0

En Google Colab, la configuración de la webcam para hacer un *feed* en tiempo real no es algo trivial, por lo que ahora nos vamos limitaremos a tomar algunas imágenes con la cámara y procesarlas por separado. Para ello, reutilizamos algunos de los *snippets* de código facilitados por el entorno:

In [ ]:
# Snippet de código para capturar imágenes con la webcam provisto por Colab

from IPython.display import display, Javascript, clear_output
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])

  with open(filename, 'wb') as f:
    f.write(binary)
    
  return filename

In [ ]:
# Loopeamos para tomar distintas fotos
for i in range(5):

  try:
    filename = take_photo(filename=f'data/photo_{i}.jpg')
    print('Saved to {}'.format(filename))
    # Show the image which was just taken.
    display(Image(filename))
    # Clear output
    clear_output()

  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

Predecimos sobre las imágenes recién capturadas:

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/photo_0.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/photo_1.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/photo_2.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/photo_3.jpg -dont_show  -thresh 0.5
imShow('predictions.jpg')

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/photo_4.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights photo.jpg -dont_show
imShow('predictions.jpg')

## En resumen

En esta notebook:

- Vimos cómo  utilizar la implementación oficial de YOLO v4 para hacer detección de objetos en imágenes.
- Presentamos los comandos básicos para poder hacer inferencia sobre imágenes estáticas, videos y *feeds* de webcam.
- Aprendimos a regular el umbral de confianza mínima exigida al modelo para hacer la detección.
- Exportamos los resultados obtenidos a archivos de distintos formatos.